In [2]:
import numpy as np
import pandas as pd 

In [5]:
df=pd.read_excel("train.xlsx")
df.head() 

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report  

In [54]:
import preprocess_kgptalkie as ps
import re 
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x  

In [40]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall  



  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to c:\users\dell\appdata\local\temp\pip-req-build-k77gbpgd
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=7677 sha256=f70dfe286d01b9092b078e9335266116d8687e97f9551f55e0f19c4425fe5531
  Stored in directory: C:\Users\Dell\AppData\Local\Temp\pip-ephem-wheel-cache-yf68ttic\wheels\fd\71\47\e04f208ad659a295ccb69022e14de7d20ccc6fc30a5a935f09
Successfully built preprocess-kgptalkie
  Attempting uninstall: preprocess-kgptalkie
    Found existing installation: preprocess-kgptalkie 0.1.3
    Uninstalling preprocess-kgptalkie-0.1.3:
      Successfully uninstalled preprocess-kgptalkie-0.1.3


  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git 'C:\Users\Dell\AppData\Local\Temp\pip-req-build-k77gbpgd'


In [42]:
!pip install spacy 


In [52]:
import spacy
from spacy.cli import download
print(download('en')) 

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✘ Couldn't link model to 'en'
Creating a symlink in spacy/data failed. Make sure you have the required
permissions and try re-running the command as admin, or use a virtualenv. You
can still import the model as a module and call its load() method, or create the
symlink manually.
C:\Users\Dell\anaconda3\lib\site-packages\en_core_web_sm -->
C:\Users\Dell\anaconda3\lib\site-packages\spacy\data\en
⚠ Download successful but linking failed
Creating a shortcut link for 'en' didn't work (maybe you don't have admin
permissions?), but you can still load the model via its full package name: nlp =
spacy.load('en_core_web_sm')
None


In [55]:
import en_core_web_sm
nlp = en_core_web_sm.load() 

In [57]:
df['Reviews']=df['Reviews'].apply(lambda x: get_clean(x)) 

In [59]:
df.head() 

,Reviews,Sentiment
0,when i first tuned in on this morning news i t...,neg
1,mere thoughts of going overboard aka babes aho...,neg
2,why does this movie fall well below standards ...,neg
3,wow and i thought that any steven segal movie ...,neg
4,the story is seen before but that doesand matt...,neg


In [61]:
tfidf=TfidfVectorizer(max_features=5000)
x=df['Reviews']
y=df['Sentiment']  
x=tfidf.fit_transform(x)

In [62]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=0)

In [63]:
clf=LinearSVC()
clf.fit(x_train,y_train)

LinearSVC()

In [65]:
x 

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2845641 stored elements in Compressed Sparse Row format>

In [70]:
y_pred=clf.predict(x_test)
y_pred

array(['pos', 'neg', 'neg', ..., 'pos', 'neg', 'neg'], dtype=object)

In [71]:
print(classification_report(y_test,y_pred)) 

              precision    recall  f1-score   support

         neg       0.87      0.87      0.87      2480
         pos       0.87      0.88      0.88      2520

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [72]:
x="" 
x=get_clean(x)
vect=tfidf.transform([x])

In [73]:
vect.shape

(1, 5000)

In [75]:
clf.predict(vect)

array(['pos'], dtype=object)

In [76]:
import pickle 
pickle.dump(clf,open('imdbSentimentalAnalycis_model','wb'))